### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [3]:
from collections import namedtuple, deque
import random
from bisect import insort
import pandas as pd
import numpy as np

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000):#10000000
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

# Среднее + дисперсия + мин + медиана + макс для nightmare_stream()

In [4]:
from decimal import *
getcontext().prec = 28
a = nightmare_stream()
a1 = nightmare_stream()
count = 0.
nightmare_result = np.zeros(6, dtype='object')
nightmare_var = np.zeros(1, dtype='object')
nightmare_mean = np.zeros(1, dtype='object')
nightmare_median = []
for j in a:
    insort(nightmare_median, j)
    nightmare_mean += (j)#Decimal
    nightmare_var += (j)**2#Decimal
    count+=1
    if count==999: break
for j in a:
    insort(nightmare_median, j)
#     print('inter_median', median)
    nightmare_mean += (j)#Decimal
    nightmare_var += (j)**2#Decimal
    count+=1
#     print('med',median[1])
    nightmare_result[3] += nightmare_median[500]
    nightmare_result[2] += nightmare_median[0]
    nightmare_result[4] += nightmare_median[-1]
    nightmare_result[0] += nightmare_mean#/1000
    nightmare_result[1] += nightmare_var#nightmare_result[1] += 1000*nightmare_var - (nightmare_mean)**2
    nightmare_result[5] += Decimal(nightmare_mean[0])**2
    for i in a1:
        nightmare_median.remove(i)
        nightmare_mean -= (i)#Decimal
        nightmare_var -= (i)**2#Decimal
        break
nightmare_result[1] =  Decimal(nightmare_result[1][0])/1000 - Decimal(nightmare_result[5])/1000000      
nightmare_result[0] /= 1000*(10000000-1000+1)
nightmare_result[1] /= (10000000-1000+1)#1000000*
nightmare_result[2] /= (10000000-1000+1)
nightmare_result[3] /= (10000000-1000+1)
nightmare_result[4] /= (10000000-1000+1)
print('mean',nightmare_result[0])
print('дисперсия',(nightmare_result[1]))
print('min:',nightmare_result[2])
print('медиана',nightmare_result[3])
print('max:',nightmare_result[4])

mean [499880345.8782329]
дисперсия 83228908564031114.58817449053
min: 1017512.2936084315
медиана 500438415.6358754
max: 999017359.9740516


# Среднее + дисперсия + мин + медиана + макс для medium_stream()

In [709]:
a = medium_stream()
a1 = medium_stream()
count = 0.
medium_result = np.zeros(6, dtype='object')
medium_mean_var = np.zeros(2, dtype='object')
medium_median = []
for j in a:
    insort(medium_median, j)
    medium_mean_var += (j,j**2)
    count+=1
    if count==999: break
for j in a:
    insort(medium_median, j)
    medium_mean_var += (j,j**2)
    count+=1
    medium_result[3] += medium_median[500]
    medium_result[2] += medium_median[0]
    medium_result[4] += medium_median[-1]
    medium_result[0] += medium_mean_var[0]
    medium_result[1] += medium_mean_var[1]
    medium_result[5] += (medium_mean_var[0])**2
    for i in a1:
        medium_median.remove(i)
        medium_mean_var -= (i,i**2)
        break
medium_result[1] =  Decimal(medium_result[1])/1000 - Decimal(medium_result[5])/1000000      
medium_result[0] /= 1000*(10000000-1000+1)
medium_result[1] /= (10000000-1000+1)
medium_result[2] /= (10000000-1000+1)
medium_result[3] /= (10000000-1000+1)
medium_result[4] /= (10000000-1000+1)
print('mean',medium_result[0])
print('дисперсия',(medium_result[1]))
print('min:',medium_result[2])
print('медиана',medium_result[3])
print('max:',medium_result[4])

mean 127.48113997058306
дисперсия 5455.173897874811393658226457
min: 0.019289827053722668
медиана 127.60172041186915
max: 254.97901320341902


# Среднее + дисперсия + мин + медиана + макс для easy_stream()

In [719]:
a = easy_stream()
a1 = easy_stream()
count = 0.
easy_result = np.zeros(6, dtype='object')
easy_mean_var = np.zeros(2, dtype='object')
easy_median = deque(maxlen=1000)
for j in a:
    easy_mean_var += (j,j**2)
    easy_median.append(j)
    count+=1
    if count>=1000:
        easy_result[3] += easy_median[500]
        easy_result[2] += easy_median[0]
        easy_result[4] += easy_median[-1]
        easy_result[0] += easy_mean_var[0]
        easy_result[1] += easy_mean_var[1]
        easy_result[5] += (easy_mean_var[0])**2
        for i in a1:
            easy_mean_var -= (i,i**2)
            break
easy_result[1] =  (easy_result[1])/1000 - (easy_result[5])/1000000      
easy_result[0] /= 1000*(10000000-1000+1)
easy_result[1] /= (10000000-1000+1)
easy_result[2] /= (10000000-1000+1)
easy_result[3] /= (10000000-1000+1)
easy_result[4] /= (10000000-1000+1)
print('mean',easy_result[0])
print('дисперсия',(easy_result[1]))
print('min:',easy_result[2])
print('медиана',easy_result[3])
print('max:',easy_result[4])

mean 4999675.276494781
дисперсия 83439.34015968195
min: 4999175.792842705
медиана 4999675.776641486
max: 5000174.759596184


## Очень долгий расчет дисперсии и среднего
# настолько долго, что и запускать не стоит

In [ ]:
# stream = nightmare_stream()
# d = deque(maxlen=1000)
# mean_list = []
# var_list = []
# count = 0
# for i in stream:
#     d.append(Decimal(i))
#     count += 1
#     if count == 999: break
# for i in stream:
#     d.append(Decimal(i))
#     count += 1
#     mean_list.append(np.mean(d))
#     var_list.append(np.var(d))
# print('mean:','{:0.2f}'.format(np.mean(mean_list)))
# print('var:','{:0.2f}'.format(np.mean(var_list)))